# Get Routes Table

In this jupyter notebook you will see how I clean data from the "ascents table", this table has a record of the climber's ascension of a route, it also contains more data like location, grade, hard/soft, rating.. Each climber fills it in their particular way, you will see there is a lot to clean. this job is divided in:
1. String cleaning for each column (using class `text_processor_class.py`)
2. Sentiment (first checks language and then if comments are positive or negative, using class `text_processor_class.py`)
3. Splitting
4. Cleaning and filtering outlayers
5. Adding climbers table features (this table was obtained from ascents table as well, how to in [this project](https://github.com/jordi-zaragoza/Climbing-Data-Analysis))
6. Clean and group naming (it compares names and replaces if they are alike, using class `text_processor_class.py`)
    - crag
    - sector
    - route name
    
If you want to know how my text_processr_class works, take a look at `GUIDE.text_processor_class.ipynb`

In [3]:
import pandas as pd 
import numpy as np
import clean_jor
import tables_jor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from text_processor_class import *

ModuleNotFoundError: No module named 'vaderSentiment'

In [4]:
pd.set_option("display.max_columns",None)
pd.set_option('display.max_rows', 10)

In [5]:
txt = text_processor()

NameError: name 'text_processor' is not defined

## 1. String cleaning

### Get conversions table

In [2]:
conversions = tables_jor.conversion_table()

NameError: name 'tables_jor' is not defined

### Get and clean ascent table

I have to create a routes table where I will select the most valuable features for each route based on user evaluation.

In [5]:
# %%time
# ascent = pd.read_csv('../../databases/ascent.csv',low_memory=False)
# ascent_df = clean_jor.Correct_grade_id(ascent)
# ascent_df.to_csv('../../databases/ascent_grade_correction.csv')
'''
CAUTION:
this piece of code above takes:
CPU times: user 38.3 s, sys: 2.55 s, total: 40.9 s
Wall time: 41.2 s
'''

ascent_df = pd.read_csv('../../databases/ascent_grade_correction.csv',low_memory=False, index_col=0)

### Choosing the correct features

In [20]:
print(ascent_df.shape)
ascent_df.head(3)

(4111878, 28)


,id,user_id,grade_id,notes,raw_notes,method_id,climb_type,total_score,date,year,last_year,rec_date,project_ascent_date,name,crag_id,crag,sector_id,sector,country,comment,rating,description,yellow_id,climb_try,repeat,exclude_from_ranking,user_recommended,chipped
0,2,1.0,36,NaN,0,3,0,545,918342000.0,1999.0,0.0,1.107126e+09,0.0,The King And I,16596.0,Railay,61.0,Dum's kitchen,THA,Thailand\n,0.0,NaN,255.0,0.0,0.0,0.0,0.0,0.0
1,3,1.0,36,NaN,0,3,0,545,925509600.0,1999.0,0.0,1.107126e+09,0.0,vet ej,0.0,Nya berg - segl.,0.0,NaN,NaN,\n,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,4,1.0,36,NaN,0,3,0,545,933026400.0,1999.0,0.0,1.107126e+09,0.0,Mr Big,209.0,Sjöända,371.0,Huvudväggen,SWE,\n,0.0,NaN,255.0,0.0,0.0,0.0,0.0,0.0


In [9]:
ascent_df2 = ascent_df.copy()
ascent_df2 = ascent_df2[ascent_df.exclude_from_ranking == 0]
ascent_df2 = ascent_df2[ascent_df.climb_type == 0] # sportsclimb
ascent_df2 = ascent_df2.drop(columns = ['raw_notes','exclude_from_ranking','climb_type','total_score','date','year','rec_date','project_ascent_date','description','method_id','last_year'])
print("2..", ascent_df2.shape)

<ipython-input-9-f6d93b26d55e>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ascent_df2 = ascent_df2[ascent_df.climb_type == 0] # sportsclimb


2.. (2851212, 17)


In [10]:
ascent_df2.head(3)

,id,user_id,grade_id,notes,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped
0,2,1.0,36,NaN,The King And I,16596.0,Railay,61.0,Dum's kitchen,THA,Thailand\n,0.0,255.0,0.0,0.0,0.0,0.0
1,3,1.0,36,NaN,vet ej,0.0,Nya berg - segl.,0.0,NaN,NaN,\n,0.0,0.0,0.0,0.0,0.0,0.0
2,4,1.0,36,NaN,Mr Big,209.0,Sjöända,371.0,Huvudväggen,SWE,\n,0.0,255.0,0.0,0.0,0.0,0.0


## 2. Clean string columns

We initialize the class text_processor and run the clinning for each column

In [11]:
# ascent_df2_2 = ascent_df2.copy()
# ascent_df2_2['name'] = txt.cleaning_function(ascent_df2_2.name)
# ascent_df2_2['sector'] = txt.cleaning_function(ascent_df2_2.sector)
# ascent_df2_2['crag'] = txt.cleaning_function(ascent_df2_2.crag)
# ascent_df2_2['country'] = txt.cleaning_function(ascent_df2_2.country)
# ascent_df2_2['comment'] = txt.cleaning_function(ascent_df2_2.comment)
'''
CAUTION:
this piece of code above takes:
CPU times: user 2min 45s, sys: 739 ms, total: 2min 46s
Wall time: 2min 47s
'''
ascent_df2_2.head(3)

,id,user_id,grade_id,notes,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped
0,2,1.0,36,NaN,the king and i,16596.0,railay,61.0,dums kitchen,tha,thailand,0.0,255.0,0.0,0.0,0.0,0.0
1,3,1.0,36,NaN,vet ej,0.0,nya berg segl,0.0,nan,nan,,0.0,0.0,0.0,0.0,0.0,0.0
2,4,1.0,36,NaN,mr big,209.0,sjoanda,371.0,huvudvaggen,swe,,0.0,255.0,0.0,0.0,0.0,0.0


## 3. Check if the coments are positive or negetive

We want this to run it once as it will take a long time

In [12]:
# %%time
# from polyglot.detect.base import logger
# # we disable the warning messages
# logger.disabled = True
# ascent_df2_2['sentiment'] = ascent_df2_2.comment.apply(lambda x: txt.comment_sentiment(x) if len(str(x))<4  else 0)
# ascent_df2_2['comment_bool'] = ascent_df2_2.comment.apply(lambda x: 0 if len(str(x))<4  else 1)
# logger.disabled = False
# ascent_df2_2.to_csv('../../databases/ascent_with_sentiment.csv')
'''
CAUTION:
this piece of code above takes:
CPU times: user 21.5 s, sys: 716 ms, total: 22.2 s
Wall time: 22.9 s
'''

ascent_df2_2 = pd.read_csv('../../databases/ascent_with_sentiment.csv',low_memory=False, index_col=0)
ascent_df2_2.head(3)

CPU times: user 1min 10s, sys: 707 ms, total: 1min 11s
Wall time: 1min 11s


,id,user_id,grade_id,notes,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped,sentiment,comment_bool
0,2,1.0,36,NaN,the king and i,16596.0,railay,61.0,dums kitchen,tha,thailand,0.0,255.0,0.0,0.0,0.0,0.0,0.0,1
1,3,1.0,36,NaN,vet ej,0.0,nya berg segl,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,4,1.0,36,NaN,mr big,209.0,sjoanda,371.0,huvudvaggen,swe,NaN,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0


##  4. Continue cleaning

Here we split notes column into easy, hard, first_ascent, traditional.. and rearrange the grading based on that (+1 if hard, -1 if soft.. that is a "/" grade, like from 7a to 7a/)

In [13]:
ascent_df2_2.notes.value_counts(dropna = False).head(5)

NaN            2128264
Second Go       265786
Soft            155117
Hard            128852
Traditional      62667
Name: notes, dtype: int64

### Splitting

In [14]:
# %%time
# #Add a bool for rating
# ascent_df2_2['rating_bool'] = ascent_df2_2.rating.apply(lambda x: 0 if x == 0  else 1)
# # Split the notes column
# ascent_df3 = ascent_df2_2.copy()
# ascent_df3.notes = ascent_df3.notes.fillna('-')
# ascent_df3.reset_index(drop = True, inplace = True)
# notes = ascent_df3.notes.apply(clean_jor.Split_notes)
# notes_df = pd.DataFrame(notes.to_list(), columns = ['first_ascent', 'soft', 'hard', 'traditional'])
# ascent_df3 = pd.concat([ascent_df3,notes_df], axis = 1)
# ascent_df3 = ascent_df3.drop(columns = ['notes'])
# ascent_df3.grade_id = ascent_df3.apply(clean_jor.Easy_hard, axis = 1)   
# print(ascent_df3.shape)
# ascent_df3.to_csv('../../databases/ascent_df3.csv')
'''
The piece of code above takes:
CPU times: user 1min 1s, sys: 1.49 s, total: 1min 2s
Wall time: 1min 3s
'''

ascent_df3 = pd.read_csv('../../databases/ascent_df3.csv',low_memory=False, index_col=0)

(2851212, 23)
CPU times: user 1min 6s, sys: 1.58 s, total: 1min 8s
Wall time: 1min 8s


In [15]:
ascent_df3.head(3)

,id,user_id,grade_id,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped,sentiment,comment_bool,rating_bool,first_ascent,soft,hard,traditional
0,2,1.0,36,the king and i,16596.0,railay,61.0,dums kitchen,tha,thailand,0.0,255.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0
1,3,1.0,36,vet ej,0.0,nya berg segl,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,4,1.0,36,mr big,209.0,sjoanda,371.0,huvudvaggen,swe,NaN,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0


### Filtering task

In [16]:
# Grade below 9c
ascent_df4 = ascent_df3.copy()
ascent_df4 = ascent_df4[ascent_df4['grade_id'] < 79] # below 9c
ascent_df4 = ascent_df4[ascent_df4['grade_id'] > 28] # # above 5a
ascent_df4.dropna(subset=['name'],inplace = True)
ascent_df4.reset_index(drop = True, inplace = True)
print("4..",ascent_df4.shape)

4.. (2737450, 23)


#### - Name column

In [17]:
ascent_df4.name.value_counts().head()

dont know name    3751
no name           3096
route             3003
unknown           2762
n                 2018
Name: name, dtype: int64

In [18]:
# filter the weird naming
ascent_df5 = ascent_df4.copy()
ascent_df5 = ascent_df5["" != ascent_df5["name"]]
ascent_df5 = ascent_df5["dont know name" != ascent_df5["name"]]
ascent_df5 = ascent_df5["no name" != ascent_df5["name"]]
ascent_df5 = ascent_df5["route" != ascent_df5["name"]]
ascent_df5 = ascent_df5["unknown" != ascent_df5["name"]]
ascent_df5 = ascent_df5["senza nome" != ascent_df5["name"]]
ascent_df5 = ascent_df5["n" != ascent_df5["name"]]
ascent_df5 = ascent_df5["via" != ascent_df5["name"]]
ascent_df5 = ascent_df5["a" != ascent_df5["name"]]
ascent_df5.name.value_counts().head(6)

dna                1584
carpe diem         1238
nirvana            1074
toma castanazo     1023
gladiator           996
billy el rapido     972
Name: name, dtype: int64

In [19]:
ascent_df5.shape

(2719302, 23)

#### - Crag column

In [20]:
ascent_df5.crag.value_counts().head(6)

frankenjura        111402
kalymnos            95413
rodellar            57326
red river gorge     50331
margalef            46931
siurana             44608
Name: crag, dtype: int64

#### - Sector column

In [22]:
ascent_df5.sector.value_counts().head(6)

odyssey            15719
sloneczne skaly    11089
el camino          10159
massone             9815
demi lune           8793
misja pec b         8329
Name: sector, dtype: int64

In [23]:
ascent_df5.sector_id.value_counts().head(6)

0.0      1036654
62.0       15718
104.0      11060
64.0        9810
63.0        8811
65.0        8736
Name: sector_id, dtype: int64

## 5. Climbers table 

We will use this table to add heigh and sex on each route of the ascent table

In [24]:
# Get the table already cleaned in the other project
climber_df = pd.read_csv('../data/climber_df.csv',low_memory=False, index_col=0)

In [25]:
climber_df = climber_df[['height','sex']].reset_index()

In [26]:
ascent_df5_2 = pd.merge(ascent_df5,climber_df,how="inner",left_on='user_id',right_on='user_id')
ascent_df5_2.head(3)

,id,user_id,grade_id,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped,sentiment,comment_bool,rating_bool,first_ascent,soft,hard,traditional,height,sex
0,2,1.0,36,the king and i,16596.0,railay,61.0,dums kitchen,tha,thailand,0.0,255.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,177,0
1,3,1.0,36,vet ej,0.0,nya berg segl,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,177,0
2,4,1.0,36,mr big,209.0,sjoanda,371.0,huvudvaggen,swe,NaN,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,177,0


I want to check if its hard or soft for tall people, so I will create a feature called tall_recomend:
- Tall gets deffined as higher than average + 5cm / small is avg - 5cm
- If you are tall and say its soft
- If you are small and say its hard

In [27]:
mean_height = ascent_df5_2.height.mean()

In [28]:
ascent_df5_2['tall_recommend'] = ascent_df5_2.apply(lambda x: clean_jor.Tall_recommend(x,mean_height), axis = 1)

In [29]:
ascent_df5_2.shape

(1402386, 26)

In [30]:
# ascent_df5_2.to_csv('../data/ascent_df5_2.csv')
# ascent_df5_2 = pd.read_csv('../data/ascent_df5_2.csv',low_memory=False, index_col=0)

# 6. Correcting and grouping crag names, sector names and route names

I want to correct the names, for that purpose I will use one the instance "txt" defined above from <b>class text_processor

## Crag names

In [31]:
countries = ascent_df5_2.country.value_counts(dropna = False).index
countries

Index(['esp', 'fra', 'usa', 'ita', 'pol', 'deu', 'grc', 'swe', 'aut', 'bra',
       ...
       'sur', 'prk', 'myt', 'hti', 'kir', 'kaz', 'hnd', 'flk', 'bdi', 'idn'],
      dtype='object', length=142)

In [32]:
crags_from_country = ascent_df5_2[ascent_df5_2.country == countries[60]].crag
crags_from_country.value_counts().head(6)

long dong        78
guanzailing      37
guan zai ling    11
guangzulin        4
gaunzailing       4
guanzaiing        1
Name: crag, dtype: int64

In [34]:
lst = txt.replace_similar_array(list(crags_from_country))
crags_from_country_new = pd.Series(lst)
crags_from_country_new.value_counts().head(6)

long dong       78
guanzailing     54
guangzulin       4
guan zai lin     1
dtype: int64

In [35]:
crags_from_country.index

Int64Index([ 156161,  156162,  156163,  156164,  156165,  156166,  475497,
             475540,  475541,  475542,
            ...
            1252980, 1252981, 1252982, 1252983, 1285380, 1391606, 1391615,
            1391616, 1391617, 1391618],
           dtype='int64', length=137)

In [36]:
ascent_copy = ascent_df5_2.copy()
ascent_copy.loc[crags_from_country.index,'crag'] = lst

In [37]:
ascent_copy.loc[156166,:].head(6)

id                  167333
user_id             4984.0
grade_id                51
name        brooben mirror
crag_id            30705.0
crag           guanzailing
Name: 156166, dtype: object

We put all together in a for loop... 

This option is very time consuming, check `GUIDE.text_processor_class`and `extra.text_process_location` for more details

In [38]:
# Pre filter
ascent_df5_2.name = ascent_df5_2.name.astype(str)
ascent_df5_2.crag = ascent_df5_2.crag.astype(str)
ascent_df5_2.sector = ascent_df5_2.sector.astype(str)

ascent_df5_2 = ascent_df5_2[ascent_df5_2.name.apply(lambda x: len(x) > 3)]
ascent_df5_2 = ascent_df5_2[ascent_df5_2.crag.apply(lambda x: len(x) > 3)]
ascent_df5_2 = ascent_df5_2[ascent_df5_2.sector.apply(lambda x: len(x) > 3)]

## Clean crag

In [42]:
# %%time
# ascent_df5_3 = ascent_df5_2.copy()
# for country in countries:
#     print(country)
#     crags_from_country = ascent_df5_3[ascent_df5_3.country == country].crag
#     lst = txt.replace_frequent_with_table(list(crags_from_country))
#     ascent_df5_3.loc[crags_from_country.index,'crag'] = lst 
# ascent_df5_3.to_csv('../data/ascent_df5_3.csv')
'''
The piece of code above takes:
CPU times: user 7min 21s, sys: 616 ms, total: 7min 21s
Wall time: 7min 23s
'''

ascent_df5_3 = pd.read_csv('../data/ascent_df5_3.csv',low_memory=False, index_col=0)

esp
fra
usa
ita
pol
deu
grc
swe
aut
bra
svn
nor
nan
che
hrv
tur
zaf
aus
can
tha
prt
gbr
bel
ukr
mex
nzl
rom
bgr
lux
rus
hun
arg
srb
chn
svk
ven
cze
fin
isr
lao
isl
col
chl
per
mar
mkd
mlt
jpn
cub
and
bih
nld
msr
reu
ecu
dnk
vnm
irn
pri
ind
twn
fxx
mys
kor
hkg
cyp
omn
jor
mdg
bol
yug
are
irl
syr
phl
cri
pan
alb
nam
egy
arm
gtm
dom
cym
zwe
fro
mne
abw
kgz
mco
swz
slv
nga
lbn
sau
sgs
smr
npl
ken
aze
bvt
grl
lka
gib
eth
khm
uga
lie
civ
bhs
geo
tjk
som
fji
glp
asm
gha
zmb
bfa
cmr
mdv
slb
tun
gum
ncl
mmr
mli
sdn
ltu
pak
atf
cpv
sur
prk
myt
hti
kir
kaz
hnd
flk
bdi
idn
CPU times: user 7min 59s, sys: 1.43 s, total: 8min 1s
Wall time: 8min 3s


In [43]:
display(ascent_df5_3.head(3))
ascent_df5_3.shape

,id,user_id,grade_id,name,crag_id,crag,sector_id,sector,country,comment,rating,yellow_id,climb_try,repeat,user_recommended,chipped,sentiment,comment_bool,rating_bool,first_ascent,soft,hard,traditional,height,sex,tall_recommend
0,2,1.0,36,the king and i,16596.0,railay,61.0,dums kitchen,tha,thailand,0.0,255.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,177,0,0
2,4,1.0,36,mr big,209.0,sjoanda,371.0,huvudvaggen,swe,NaN,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,177,0,0
3,5,1.0,36,tak ska du ha,209.0,sjoanda,371.0,huvudvaggen,swe,NaN,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,177,0,0


(1013951, 26)

### Group crag

In [44]:
# multiple columns
ascent_df6 = ascent_df5_2.copy()
ascent_df6 = ascent_df6.groupby(['country','crag','sector','name']).agg( 
    ascents_count = ('grade_id', 'count'),
    grade_sum = ('grade_id', 'sum'),
    sentiment_sum = ('sentiment','sum'),
    sentiment_count = ('comment_bool','sum'),
    rating_sum = ('rating', 'sum'),
    rating_count = ('rating_bool', 'sum'),
    repeat_sum = ('repeat','sum'),
    recommend_sum = ('user_recommended','sum'),
    chiped_sum = ('chipped','sum'),
    soft_sum = ('soft','sum'),
    hard_sum = ('hard','sum'),
    traditional_sum = ('traditional','sum'),
    tall_recommend_sum = ('tall_recommend','sum'),
    sex_sum = ('sex','sum'),
    yellow_id_sum = ('yellow_id','sum'),
    first_ascent_sum = ('first_ascent','sum'))

ascent_df6.reset_index(inplace = True)
print(ascent_df6.shape)
ascent_df6.head(5)

(276037, 20)


,country,crag,sector,name,ascents_count,grade_sum,sentiment_sum,sentiment_count,rating_sum,rating_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,sex_sum,yellow_id_sum,first_ascent_sum
0,abw,adlitzgraben,lari fari,fenrir,10,489,0.0,3,17.0,7,0.0,0.0,0.0,1,0,0,0,0,0.0,0
1,alb,atxarte,amilla,plaka desplome,1,49,0.0,0,3.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0
2,alb,atxarte,la puerta,andrakila,1,44,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0
3,alb,atxarte,la puerta,la ley del bombero,1,49,0.0,0,1.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0
4,alb,atxarte,la puerta,la napa se escapa,1,46,0.0,0,1.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0


In [45]:
# ascent_df6.to_csv('../data/ascent_df6.csv')
# ascent_df6 = pd.read_csv('../data/ascent_df6.csv',low_memory=False, index_col=0)

In [46]:
ascent_df6[ascent_df6.country == 'esp'].crag.value_counts()

montserrat               2902
margalef                 2355
la pedriza               2273
siurana                  1998
rodellar                 1873
                         ... 
puig st marti               1
fragas del boeza            1
fotosintesis                1
raco de farol borriol       1
las cimbras                 1
Name: crag, Length: 2124, dtype: int64

## Clean sector

In [47]:
## Prefilter
counts = ascent_df6.sector.value_counts()[ascent_df6.sector.value_counts() > 20] # At least 20 routes per sector
ascent_df6 = ascent_df6[ascent_df6.sector.apply(lambda x: x in list(counts.index))]

In [48]:
display(ascent_df6.head(3))
ascent_df6.shape

,country,crag,sector,name,ascents_count,grade_sum,sentiment_sum,sentiment_count,rating_sum,rating_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,sex_sum,yellow_id_sum,first_ascent_sum
13,alb,ibiza,anfiteatro,los pajaros de hamlet,1,44,0.0,0,3.0,1,0.0,0.0,0.0,0,0,0,0,0,255.0,0
15,alb,margalef,can torxa,escalagmita,1,38,0.0,1,3.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0
16,alb,navire,tarn,lequipeur desagrege,1,51,0.0,0,2.0,1,1.0,0.0,0.0,0,0,0,0,0,0.0,0


(120878, 20)

In [49]:
# %%time
# ascent_df7 = ascent_df6.copy()
# for country in countries:
#     print(country)
#     crags = ascent_df7[ascent_df7.country == country].crag.value_counts(dropna = False).index
    
#     for crag in crags:  
#         sectors_from_crag = ascent_df7[ascent_df7.crag == crag].sector       
#         lst = txt.replace_frequent_with_table(list(sectors_from_crag), show = False)
#         ascent_df7.loc[sectors_from_crag.index,'sector'] = lst 
        
# ascent_df7.to_csv('../data/ascent_df7.csv')
'''
The piece of code above takes:
CPU times: user 1min 17s, sys: 116 ms, total: 1min 17s
Wall time: 1min 17s
'''

ascent_df7 = pd.read_csv('../data/ascent_df7.csv',low_memory=False, index_col=0)

esp
fra
usa
ita
pol
deu
grc
swe
aut
bra
svn
nor
nan
che
hrv
tur
zaf
aus
can
tha
prt
gbr
bel
ukr
mex
nzl
rom
bgr
lux
rus
hun
arg
srb
chn
svk
ven
cze
fin
isr
lao
isl
col
chl
per
mar
mkd
mlt
jpn
cub
and
bih
nld
msr
reu
ecu
dnk
vnm
irn
pri
ind
twn
fxx
mys
kor
hkg
cyp
omn
jor
mdg
bol
yug
are
irl
syr
phl
cri
pan
alb
nam
egy
arm
gtm
dom
cym
zwe
fro
mne
abw
kgz
mco
swz
slv
nga
lbn
sau
sgs
smr
npl
ken
aze
bvt
grl
lka
gib
eth
khm
uga
lie
civ
bhs
geo
tjk
som
fji
glp
asm
gha
zmb
bfa
cmr
mdv
slb
tun
gum
ncl
mmr
mli
sdn
ltu
pak
atf
cpv
sur
prk
myt
hti
kir
kaz
hnd
flk
bdi
idn
CPU times: user 1min 11s, sys: 220 ms, total: 1min 11s
Wall time: 1min 11s


In [50]:
print(ascent_df7.shape)
ascent_df7.head(3)

(120878, 20)


,country,crag,sector,name,ascents_count,grade_sum,sentiment_sum,sentiment_count,rating_sum,rating_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,sex_sum,yellow_id_sum,first_ascent_sum
13,alb,ibiza,anfiteatro,los pajaros de hamlet,1,44,0.0,0,3.0,1,0.0,0.0,0.0,0,0,0,0,0,255.0,0
15,alb,margalef,can torxa,escalagmita,1,38,0.0,1,3.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0
16,alb,navire,tarn,lequipeur desagrege,1,51,0.0,0,2.0,1,1.0,0.0,0.0,0,0,0,0,0,0.0,0


### Group sector

In [51]:
# We group again
# multiple columns
ascent_df9 = ascent_df7.copy()
ascent_df9 = ascent_df9.groupby(['country','crag','sector','name']).agg( 
    ascents_count = ('ascents_count', 'sum'),
    grade_sum = ('grade_sum', 'sum'), 
    sentiment_sum = ('sentiment_sum','sum'),
    sentiment_count = ('sentiment_count','sum'),
    
    rating_sum = ('rating_sum', 'sum'),
    rating_count = ('rating_count','sum'),
    
    repeat_sum = ('repeat_sum','sum'),
    recommend_sum = ('recommend_sum','sum'),
    chiped_sum = ('chiped_sum','sum'),
    soft_sum = ('soft_sum','sum'),
    hard_sum = ('hard_sum','sum'),
    traditional_sum = ('traditional_sum','sum'),
    tall_recommend_sum = ('tall_recommend_sum','sum'),
    sex_sum = ('sex_sum','sum'),
    
    yellow_id_sum = ('yellow_id_sum','sum'),
    first_ascent_sum = ('first_ascent_sum','sum'))

# # Filter more than 3 ascents per route
ascent_df9 = ascent_df9[ascent_df9.ascents_count > 1]
ascent_df9.reset_index(inplace = True)

ascent_df9.reset_index(inplace = True)

In [52]:
print(ascent_df9.shape)

(58370, 21)


Now we do the cleaning for the <b>Route names..

## Clean route names

In [53]:
# %%time
# ascent_df10 = ascent_df9.copy()
# for country in countries:
#     print(country)
#     crags = ascent_df10[ascent_df10.country == country].crag.value_counts(dropna = False).index
    
#     for crag in crags:  
#         sectors = ascent_df10[ascent_df10.crag == crag].sector.value_counts(dropna = False).index
        
#         for sector in sectors:           
#             names_from_sector = ascent_df10[ascent_df10.sector == sector].name
#             lst = txt.replace_frequent_with_table(list(names_from_sector), show = False)
#             ascent_df10.loc[names_from_sector.index,'name'] = lst 
        
# ascent_df10.to_csv('../data/ascent_df10.csv')
'''
The piece of code above takes:
CPU times: user 4min 15s, sys: 48.1 ms, total: 4min 15s
Wall time: 4min 15s
'''

ascent_df10 = pd.read_csv('../data/ascent_df10.csv',low_memory=False, index_col=0)

esp
fra
usa
ita
pol
deu
grc
swe
aut
bra
svn
nor
nan
che
hrv
tur
zaf
aus
can
tha
prt
gbr
bel
ukr
mex
nzl
rom
bgr
lux
rus
hun
arg
srb
chn
svk
ven
cze
fin
isr
lao
isl
col
chl
per
mar
mkd
mlt
jpn
cub
and
bih
nld
msr
reu
ecu
dnk
vnm
irn
pri
ind
twn
fxx
mys
kor
hkg
cyp
omn
jor
mdg
bol
yug
are
irl
syr
phl
cri
pan
alb
nam
egy
arm
gtm
dom
cym
zwe
fro
mne
abw
kgz
mco
swz
slv
nga
lbn
sau
sgs
smr
npl
ken
aze
bvt
grl
lka
gib
eth
khm
uga
lie
civ
bhs
geo
tjk
som
fji
glp
asm
gha
zmb
bfa
cmr
mdv
slb
tun
gum
ncl
mmr
mli
sdn
ltu
pak
atf
cpv
sur
prk
myt
hti
kir
kaz
hnd
flk
bdi
idn
CPU times: user 3min 53s, sys: 47 ms, total: 3min 53s
Wall time: 3min 53s


### Group route names

In [58]:
# We group again
# multiple columns
ascent_df11 = ascent_df10.copy()
ascent_df11 = ascent_df11.groupby(['country','crag','sector','name']).agg( 
    ascents_count = ('ascents_count', 'sum'),
    grade_sum = ('grade_sum', 'sum'), 
    sentiment_sum = ('sentiment_sum','sum'),
    sentiment_count = ('sentiment_count','sum'),
    
    rating_sum = ('rating_sum', 'sum'),
    rating_count = ('rating_count','sum'),
    
    repeat_sum = ('repeat_sum','sum'),
    recommend_sum = ('recommend_sum','sum'),
    chiped_sum = ('chiped_sum','sum'),
    soft_sum = ('soft_sum','sum'),
    hard_sum = ('hard_sum','sum'),
    traditional_sum = ('traditional_sum','sum'),
    tall_recommend_sum = ('tall_recommend_sum','sum'),
    sex_sum = ('sex_sum','sum'),
    
    yellow_id_sum = ('yellow_id_sum','sum'),
    first_ascent_sum = ('first_ascent_sum','sum'))

# # Filter more than 3 ascents per route
ascent_df11 = ascent_df11[ascent_df11.ascents_count > 1]
ascent_df11.reset_index(inplace = True)

In [59]:
print(ascent_df11.shape)

(55087, 20)


In [60]:
ascent_df11

,country,crag,sector,name,ascents_count,grade_sum,sentiment_sum,sentiment_count,rating_sum,rating_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,sex_sum,yellow_id_sum,first_ascent_sum
0,and,montserrat,prohibitivo,el mehir,6,294,0.0,3,10.0,5,0.0,1.0,0.0,1,1,0,-1,0,0.0,0
1,and,montserrat,prohibitivo,el pas de la discordia,5,245,0.0,4,7.0,4,0.0,1.0,0.0,0,0,0,0,0,0.0,0
2,and,montserrat,prohibitivo,la x,4,197,0.0,2,6.0,3,0.0,0.0,0.0,0,1,0,-1,0,0.0,0
3,and,tartareu,bombo suis,tenedor libre,3,133,0.0,2,3.0,1,0.0,1.0,0.0,0,1,0,0,0,0.0,0
4,arg,bandurrias,rincon,tendinitis,2,97,0.0,1,1.0,1,0.0,0.0,0.0,1,0,0,1,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55082,zaf,van stadens,the beach,bush biscuit,2,88,0.0,0,0.0,0,0.0,1.0,0.0,0,0,0,0,0,0.0,0
55083,zaf,van stadens,the beach,funky chicken,2,76,0.0,1,1.0,1,0.0,0.0,0.0,0,0,0,0,0,0.0,0
55084,zaf,van stadens,the beach,out of control,2,72,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0
55085,zaf,van stadens,the beach,sheathless,2,79,0.0,1,2.0,1,0.0,0.0,0.0,0,1,0,-1,0,510.0,0


#### Create new features and remove others...

In [61]:
# Sex sum is not interesting, but sex_sum/ascents_count it gets the ratio men/wmen climbing that route 0men/1women

ascent_df11['grade_mean'] = ascent_df11.grade_sum/ascent_df11.ascents_count
ascent_df11['sentiment_mean'] = ascent_df11.sentiment_sum/ascent_df11.sentiment_count
ascent_df11['rating_mean'] = ascent_df11.rating_sum/ascent_df11.rating_count
ascent_df11['sex_ratio'] = ascent_df11.sex_sum/ascent_df11.ascents_count
ascent_df11['yellow_id_mean'] = ascent_df11.yellow_id_sum/ascent_df11.ascents_count
ascent_df11['first_ascent_mean'] = ascent_df11.first_ascent_sum/ascent_df11.ascents_count

ascent_df11 = ascent_df11.drop(columns = ['grade_sum','sentiment_sum','sentiment_count',
                                          'rating_sum','rating_count','sex_sum','yellow_id_sum','first_ascent_sum'])

print(ascent_df11.shape)
ascent_df11.tail(5)

(55087, 18)


,country,crag,sector,name,ascents_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,grade_mean,sentiment_mean,rating_mean,sex_ratio,yellow_id_mean,first_ascent_mean
55082,zaf,van stadens,the beach,bush biscuit,2,0.0,1.0,0.0,0,0,0,0,44.0,NaN,NaN,0.0,0.0,0.0
55083,zaf,van stadens,the beach,funky chicken,2,0.0,0.0,0.0,0,0,0,0,38.0,0.0,1.0,0.0,0.0,0.0
55084,zaf,van stadens,the beach,out of control,2,0.0,0.0,0.0,0,0,0,0,36.0,NaN,NaN,0.0,0.0,0.0
55085,zaf,van stadens,the beach,sheathless,2,0.0,0.0,0.0,0,1,0,-1,39.5,0.0,2.0,0.0,255.0,0.0
55086,zaf,waterval boven,baboon buttress,pretenders,2,0.0,1.0,0.0,0,0,0,0,49.0,0.0,NaN,0.0,255.0,0.0


The routes will be unique, but we only have the route_name as Id, I will asign a route_id to each route based on the name and the crag_id:

In [62]:
ascent_df12 = ascent_df11.copy()
ascent_df12.index.names = ['name_id']
ascent_df12.reset_index(inplace=True)

In [63]:
# And I will create the features table
routes_features = ascent_df12

In [64]:
routes_features.to_csv('../data/routes_features.csv')

In [74]:
import climb_jor
climb_jor.Route_search(ascent_df11,'oriol')

,country,crag,sector,name,ascents_count,repeat_sum,recommend_sum,chiped_sum,soft_sum,hard_sum,traditional_sum,tall_recommend_sum,grade_mean,sentiment_mean,rating_mean,sex_ratio,yellow_id_mean,first_ascent_mean
19058,esp,montserrat,can jorba,oriol,8,0.0,2.0,0.0,1,0,0,-1,58.875,0.0,2.857143,0.125,255.0,0.0
19454,esp,montserrat,el vermell,oriol,5,0.0,1.0,0.0,0,0,0,0,59.000,0.0,2.500000,0.200,255.0,0.0
